In [1]:
from __future__ import print_function, unicode_literals, absolute_import, division
import numpy as np
from itertools import islice 
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import os
import glob
from tifffile import imread
from csbdeep.utils import Path, normalize
from skimage.segmentation import find_boundaries
from csbdeep.io import save_tiff_imagej_compatible
from stardist import dist_to_coord, non_maximum_suppression, polygons_to_label
from stardist import random_label_cmap, draw_polygons, sample_points,fill_label_holes
from stardist import Config, StarDist

np.random.seed(6)
lbl_cmap = random_label_cmap()

/data/u934/service_imagerie/v_kapoor/anaconda2/envs/tensorflowpy3pt5/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/data/u934/service_imagerie/v_kapoor/anaconda2/envs/tensorflowpy3pt5/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
/data/u934/service_imagerie/v_kapoor/anaconda2/envs/tensorflowpy3pt5/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
basedir = '/data/u934/service_imagerie/v_kapoor/HEARD/UNet-training/test3D/'
basedirResultsWithoutShape = '/data/u934/service_imagerie/v_kapoor/HEARD/UNet-training/results3D/'
ModelNameWithoutShape = 'StemCellWithoutShapeCompletion'
BaseDirModels = 'models'

In [12]:
Path = os.path.join(basedir, '*.tif')

Y = []
Names = []

filesRaw = glob.glob(Path)
maxtime = 0
for fname in filesRaw:
     x = imread(fname)
     Y.append(x)
     Names.append(fname)
      
Y.sort
Names.sort


threshold = 0.6

axes = 'YX'
print('Total number of images to be segmented = ', len(Y))
print('Image size = ', Y[0].shape)


Total number of images to be segmented =  1
Image size =  (73, 512, 512)


In [13]:
model_no_sc = StarDist(None, name = ModelNameWithoutShape, basedir = BaseDirModels )

Loading network weights from 'weights_best.h5'.


In [ ]:
img = normalize(Y[0],1,99.8)
prob = np.array(img)
labels = np.array(img)


dist = np.zeros((img.shape[0], img.shape[1] ,img.shape[2], 32))
coord = np.zeros((img.shape[0], img.shape[1] ,img.shape[2],2, 32))


maxpoints = 0;
secdim = 0;
for i in range(timerange):
      smallimg = img[i,:,:]
      smallprob, smalldist = model_no_sc.predict(smallimg)
      smallcoord = dist_to_coord(smalldist)
      smallpoints = non_maximum_suppression(smallcoord,smallprob,prob_thresh=threshold)
      secdim = smallpoints.shape[1]
      if smallpoints.shape[0] > maxpoints: 
          maxpoints = smallpoints.shape[0]
            
            
points = np.zeros((img.shape[0],maxpoints ,2), dtype = 'i')           
newsmallpoints = np.zeros((maxpoints, secdim), dtype = 'i')
for i in range(timerange):
   
   smallimg = img[i,:,:]
   smallprob, smalldist = model_no_sc.predict(smallimg)
   smallcoord = dist_to_coord(smalldist)
   smallpoints = non_maximum_suppression(smallcoord,smallprob,prob_thresh=threshold)

   newsmallpoints[:smallpoints.shape[0], :smallpoints.shape[1]] = smallpoints;
   
   smalllabels = polygons_to_label(smallcoord,smallprob,newsmallpoints)
  
   prob[i,:,:] = smallprob[:,:] 
   labels[i,:,:] = smalllabels[:,:]
   dist[i,:,:,:] = smalldist[:,:,:]
   coord[i,:,:,:,:] = smallcoord[:,:,:,:]
    
            
    
   points[i,:,:] = newsmallpoints[:,:] 
  
  

In [ ]:
newsmalllabels = np.array(smalllabels)
newsmallprob = np.array(smallprob)
newsmalldist = np.array(smalldist)
newsmallimg = np.array(smallimg)
newsmallcoord = np.array(smallcoord)
i = timerange - 1

newsmalldist  = dist[i,:,:,:]
newsmalllabels = labels[i,:,:] 
newsmallprob = prob[i,:,:] 
newsmallimg = img[i,:,:]
newsmallcoord = coord[i,:,:,:,:]


In [ ]:
newsmalllabels = np.array(smalllabels)
newsmallprob = np.array(smallprob)
newsmalldist = np.array(smalldist)
newsmallimg = np.array(smallimg)
newsmallcoord = np.array(smallcoord)
newsmallpoints = np.array(smallpoints, dtype = 'i')
for i in range(timerange):
      
      newsmalldist  = dist[i,:,:,:]
      newsmalllabels = labels[i,:,:] 
      newsmallprob = prob[i,:,:] 
      newsmallimg = img[i,:,:]
      newsmallpoints = points[i,:,:] 
      newsmallcoord = coord[i,:,:,:,:]
    

None;



In [14]:
 def ApplyAll(model, img, timerange):
    for j in range(timerange):
      smallimg = img[j,:,:]
      smallprob, smalldist = model_no_sc.predict(smallimg)
      smallcoord = dist_to_coord(smalldist)
      smallpoints = non_maximum_suppression(smallcoord,smallprob,prob_thresh=threshold)
   
      smalllabels = polygons_to_label(smallcoord,smallprob,smallpoints)
      if j == 0:
       plt.figure(figsize=(13,12))
       points_rnd = sample_points(200,smallprob>threshold)
       numberA = 121
       numberB = 122 
       plt.subplot(numberA); plt.imshow(smallimg,cmap='gray'); 
       draw_polygons(smallcoord,smallprob,points_rnd,cmap=lbl_cmap)
       plt.axis('off'); plt.title('Polygons randomly sampled')
      
       plt.subplot(numberB); plt.imshow(smallimg,cmap='gray'); draw_polygons(smallcoord,smallprob,smallpoints,cmap=lbl_cmap)
       plt.axis('off'); plt.title('Polygons after non-maximum suppression')
       plt.tight_layout()

        
    return smalllabels
    
    
        
        
      
  

In [15]:

for i in range(0, len(Y)):

 img = normalize(Y[i],1,99.8)
 timerange = Y[i].shape[0] 
 labels = ApplyAll(model_no_sc,img, timerange)   

        
 

 save_tiff_imagej_compatible((basedirResultsWithoutShape +'Star'+ os.path.basename(Names[i])) , labels, axes)   


/data/u934/service_imagerie/v_kapoor/anaconda2/envs/tensorflowpy3pt5/lib/python3.5/site-packages/csbdeep/data/prepare.py:284: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return x[crop]
/data/u934/service_imagerie/v_kapoor/anaconda2/envs/tensorflowpy3pt5/lib/python3.5/site-packages/csbdeep/utils/utils.py:209: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  x = x[slices]


NameError: name 'smallprob' is not defined

<Figure size 936x864 with 0 Axes>